In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

## Tiền xử lý dữ liệu

In [2]:
df = pd.read_csv("data\RecipeNLG_dataset.csv", index_col=0)
df.head(2)

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."


In [3]:
df = df.drop(columns=['link', 'source'])
df.head(2)

,title,ingredients,directions,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""beef"", ""chicken breasts"", ""cream of mushroom..."


In [ ]:
len_df = len(df)
for col in df.columns:
    print(f"{col}: {len_df - df[col].count()}")
df.dropna(axis=0)

title: 0
ingredients: 0
directions: 0
NER: 0


In [4]:
df['NER'] = df['NER'].str.lower().replace(r"[^a-z, ]", "", regex=True)
df.head()

,title,ingredients,directions,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","brown sugar, milk, vanilla, nuts, butter, bite..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","beef, chicken breasts, cream of mushroom soup,..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","frozen corn, cream cheese, butter, garlic powd..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","chicken, chicken gravy, cream of mushroom soup..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","peanut butter, graham cracker crumbs, butter, ..."


In [5]:
len_df = len(df)
len_df

2231142

## Mã hóa nguyên liệu và lưu vào vector store

In [6]:
from sentence_transformers import SentenceTransformer
import chromadb

d:\python-workspace\cooksmart-website\cooksmart-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
client = chromadb.PersistentClient(path="vector-store")
collection = client.get_or_create_collection(name="NER_embed", metadata={"id_counter":0})
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [8]:
df_embed = pd.DataFrame()
df_embed

""


In [ ]:
# Mã hóa tuần tự tránh tràn ram, chỉ mã hóa 100000 bản ghi một lần
STEP = 1000
count = collection.count()
count_end = count + STEP
end = 100000
while count < end:
    print("="*75)
    print(f"BẮT ĐẦU EMBEDDING BẢN GHI TỪ {count} ĐẾN {count_end}")
    df_temp = df.iloc[count:count_end,:].copy()
    df_temp['ner_embed'] = df_temp['NER'].apply(model.encode)
    df_temp['ner_embed'] = df_temp['ner_embed'].apply(lambda x: x.astype(float).tolist())
    collection.add(
        ids = np.arange(count, count_end).astype(str).tolist(),
        embeddings=df_temp['ner_embed'].to_list(),
        metadatas=df_temp[['title', 'NER']].to_dict(orient='records'),
        documents=df_temp['title'].to_list()
    )
    print("Thêm vào vector store thành công!")
    df_embed = pd.concat([df_embed ,df_temp[['ner_embed']]])
    count = count_end
    count_end = (count_end + 1000) if (count_end + 1000) < end else end
collection.metadata["id_counter"] = end-1

BẮT ĐẦU EMBEDDING BẢN GHI TỪ 60000 ĐẾN 61000
Thêm vào vector store thành công!
BẮT ĐẦU EMBEDDING BẢN GHI TỪ 61000 ĐẾN 62000
Thêm vào vector store thành công!
BẮT ĐẦU EMBEDDING BẢN GHI TỪ 62000 ĐẾN 63000
Thêm vào vector store thành công!
BẮT ĐẦU EMBEDDING BẢN GHI TỪ 63000 ĐẾN 64000
Thêm vào vector store thành công!
BẮT ĐẦU EMBEDDING BẢN GHI TỪ 64000 ĐẾN 65000
Thêm vào vector store thành công!
BẮT ĐẦU EMBEDDING BẢN GHI TỪ 65000 ĐẾN 66000
Thêm vào vector store thành công!
BẮT ĐẦU EMBEDDING BẢN GHI TỪ 66000 ĐẾN 67000
Thêm vào vector store thành công!
BẮT ĐẦU EMBEDDING BẢN GHI TỪ 67000 ĐẾN 68000
Thêm vào vector store thành công!
BẮT ĐẦU EMBEDDING BẢN GHI TỪ 68000 ĐẾN 69000
Thêm vào vector store thành công!
BẮT ĐẦU EMBEDDING BẢN GHI TỪ 69000 ĐẾN 70000
Thêm vào vector store thành công!
BẮT ĐẦU EMBEDDING BẢN GHI TỪ 70000 ĐẾN 71000
Thêm vào vector store thành công!
BẮT ĐẦU EMBEDDING BẢN GHI TỪ 71000 ĐẾN 72000
Thêm vào vector store thành công!
BẮT ĐẦU EMBEDDING BẢN GHI TỪ 72000 ĐẾN 73000
Thêm và

In [ ]:
new_df = df.iloc[:100000,:]
len(new_df)

100000

In [46]:
new_df['ner_embed'] = df_embed
new_df.head()

,title,ingredients,directions,NER,ner_embed
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","brown sugar, milk, vanilla, nuts, butter, bite...","[-0.0064637972973287106, 0.002776391338557005,..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","beef, chicken breasts, cream of mushroom soup,...","[0.009379641152918339, 0.030905257910490036, 0..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","frozen corn, cream cheese, butter, garlic powd...","[0.03813198581337929, 0.018059231340885162, 0...."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","chicken, chicken gravy, cream of mushroom soup...","[0.02421010099351406, 0.06354163587093353, 0.0..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","peanut butter, graham cracker crumbs, butter, ...","[0.014317532069981098, 0.03072681836783886, 0...."


In [51]:
new_df.to_csv("data/RecipeNLGEmbed100000.csv")

## Test thử chức năng tìm kiếm

In [55]:
new_df['NER'][0]

'brown sugar, milk, vanilla, nuts, butter, bite size shredded rice biscuits'

In [56]:
ur_ingredients = "brown sugar, milk, vanilla, nuts, butter, bite size shredded rice biscuits"
ur_ingredients_embed = model.encode(ur_ingredients)
result = collection.query(
    query_embeddings=[ur_ingredients_embed],
    n_results=3,
    include=['metadatas', 'documents']
)
result

{'ids': [['0', '89012', '28609']],
 'embeddings': None,
 'documents': [['No-Bake Nut Cookies',
   'No-Bake Butterscotch Cookies',
   'No Bake Peanutty Drops']],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [[{'title': 'No-Bake Nut Cookies',
    'NER': 'brown sugar, milk, vanilla, nuts, butter, bite size shredded rice biscuits'},
   {'title': 'No-Bake Butterscotch Cookies',
    'NER': 'milk, butter, graham cracker crumbs, vanilla, brown sugar, peanut butter, nuts'},
   {'title': 'No Bake Peanutty Drops',
    'NER': 'brown sugar, light corn syrup, rice krispies, peanut butter, vanilla'}]],
 'distances': None}

In [60]:
result['ids'][0]

['0', '89012', '28609']